In [2]:
# using ast.literal_eval()
import sys
import ast
import os
import time
import pandas as pd
import pickle
import glob
import numpy as np
import statistics
from scipy.stats import mannwhitneyu
import math

In [3]:
with open("STRING_paths.dictionary", 'rb') as config_dictionary_file:
 
    # Step 3
    paths = pickle.load(config_dictionary_file)

In [4]:
str_paths = []


for key,val in paths.items() :
    for keys,vals in val.items():
        str_paths.append(vals)

In [5]:
statistics.mean(str_paths)

1.9846288343923668

In [3]:
# create list
file2 = "common_nodes_hPPIN_STRING_PANCAN.txt"
combined_list = []
with open(file2) as IN:
    for line in IN:
        combined_list.append(line.rstrip().split("\t")[0])

In [4]:
def compute_sp(list1, list2, paths):
    total_sp = []
    for i in range(len(list1)):
        for j in range(len(list2)):
            if(list1[i] in paths.keys()):
                shortest_paths = paths[list1[i]]
                if(list2[j] in shortest_paths.keys()):
                    total_sp.append(shortest_paths[list2[j]])
    return({"distribution": total_sp,
            "mean": statistics.mean(total_sp),
            "median": statistics.median(total_sp)})

In [5]:
# Read Cancersea Data
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/hallmark"

filenames = os.listdir(folder_path)

hallmark = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hallmark[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hallmark[filename[:-4]].append(line.rstrip())

In [6]:
# read in HP sets
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets/"

filenames = os.listdir(folder_path)

hp_genesets = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hp_genesets[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hp_genesets[filename[:-4]].append(line.rstrip().split("\t")[0])

In [7]:
hallmark = hp_genesets

In [8]:
# read in HP matched controls set
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/"

filenames = glob.glob("/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/STRING*")
hp_genesets_matched = {}

for filename in filenames:
    name = filename.split("STRING_")[1][:-4]
    hp_genesets_matched[name] = list()

    with open(filename) as IN:
        for line in IN:
            hp_genesets_matched[name].append(line.rstrip().split("\t")[0])

In [31]:
a_mean = np.zeros(shape=(len(hp_genesets), len(hallmark)))
a_median = np.zeros(shape=(len(hp_genesets), len(hallmark)))
a_dist = pd.DataFrame(index = range(len(hallmark)),
                      columns = range(len(hp_genesets)))

In [32]:
i=0
for key1,value1 in hp_genesets.items():
    j=0
    for key2,value2 in hallmark.items():
        current_info = compute_sp(value1, value2, paths)
        a_mean[i][j] = current_info["mean"]
        a_median[i][j] = current_info["median"]
        a_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [33]:
pd.DataFrame(a_median)

,0,1,2,3,4,5,6,7,8
0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
5,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
6,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
7,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [34]:
pd.DataFrame(a_mean)

,0,1,2,3,4,5,6,7,8
0,1.863501,1.918239,1.892708,1.849380,1.847338,1.891838,1.887901,1.879078,1.922620
1,1.918239,1.920146,1.931540,1.909091,1.890152,1.929169,1.921176,1.911072,1.925350
2,1.892708,1.931540,1.901548,1.874628,1.863423,1.908672,1.906556,1.899558,1.937587
3,1.849380,1.909091,1.874628,1.784741,1.801920,1.877734,1.872591,1.855436,1.918321
4,1.847338,1.890152,1.863423,1.801920,1.763095,1.868815,1.857857,1.835054,1.900133
5,1.891838,1.929169,1.908672,1.877734,1.868815,1.905834,1.905010,1.897608,1.930624
6,1.887901,1.921176,1.906556,1.872591,1.857857,1.905010,1.894760,1.890168,1.923153
7,1.879078,1.911072,1.899558,1.855436,1.835054,1.897608,1.890168,1.851797,1.916416
8,1.922620,1.925350,1.937587,1.918321,1.900133,1.930624,1.923153,1.916416,1.917051


In [35]:
pd.DataFrame(a_mean, columns = hallmark.keys(), index = hp_genesets.keys()).to_csv("STRING_meanSP_HP_Phenotypes_hallmark_9by9.csv")

In [36]:
a_matched_mean = np.zeros(shape=(len(hp_genesets_matched), len(hallmark)))
a_matched_median = np.zeros(shape=(len(hp_genesets_matched), len(hallmark)))
a_matched_dist = pd.DataFrame(index = range(len(hallmark)),
                              columns = range(len(hp_genesets_matched)))

In [37]:
i=0
for key1,value1 in hp_genesets_matched.items():
    j=0
    for key2,value2 in hallmark.items():
        current_info = compute_sp(value1, value2, paths)
        a_matched_mean[i][j] = current_info["mean"]
        a_matched_median[i][j] = current_info["median"]
        a_matched_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [38]:
pd.DataFrame(a_matched_median)

,0,1,2,3,4,5,6,7,8
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
6,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
7,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [39]:
pd.DataFrame(a_matched_mean)

,0,1,2,3,4,5,6,7,8
0,2.941028,2.951569,2.951615,2.933607,2.914537,2.953432,2.951380,2.927036,2.956053
1,2.947030,2.956644,2.956432,2.942901,2.922450,2.959359,2.957640,2.934974,2.961585
2,2.937334,2.949167,2.948782,2.929315,2.910873,2.950683,2.948613,2.923749,2.954588
3,2.960865,2.967520,2.967315,2.962716,2.939335,2.971853,2.970656,2.951470,2.971858
4,2.940015,2.950928,2.950859,2.932460,2.913557,2.952738,2.950694,2.926168,2.955768
5,2.941082,2.952080,2.951783,2.934600,2.915374,2.954146,2.952234,2.928125,2.957502
6,2.936166,2.948031,2.947796,2.927219,2.909091,2.949301,2.947119,2.921882,2.953034
7,2.975852,2.980765,2.979496,2.987044,2.960037,2.987466,2.987266,2.972541,2.987390
8,2.930880,2.943594,2.943595,2.919087,2.902166,2.944194,2.941744,2.914953,2.948383


In [40]:
pd.DataFrame(a_matched_mean, columns = hallmark.keys(), index = hp_genesets_matched.keys()).to_csv("STRING_meanSP_HPmatched_Phenotypes_hallmark_9by9.csv")

In [41]:
mannwhitneyu_p = np.zeros(shape=(len(hp_genesets), len(hallmark)))
test_dist = np.zeros(shape=(len(hp_genesets), len(hallmark)))
test_matched_dist = np.zeros(shape=(len(hp_genesets), len(hallmark)))

In [42]:
for i in range(len(hp_genesets)):
    for j in range(len(hallmark)):
        actual_dist = a_dist[i][j]
        control_dist = a_matched_dist[i][j]
        res = mannwhitneyu(actual_dist, control_dist, alternative="less")
        mannwhitneyu_p[i][j] = res.pvalue
        test_dist[i][j] = statistics.mean(actual_dist)
        test_matched_dist[i][j] = statistics.mean(control_dist)

In [43]:
hallmark.keys()

dict_keys(['downregulated.wound', 'upregulated.stress', 'downregulated.regen', 'msigdb.wound', 'msigdb.regen', 'upregulated.regen', 'upregulated.wound', 'msigdb.stress', 'downregulated.stress'])

In [44]:
pvals = pd.DataFrame(mannwhitneyu_p, columns = hallmark.keys(), index = hp_genesets.keys())

In [45]:
pvals

,downregulated.wound,upregulated.stress,downregulated.regen,msigdb.wound,msigdb.regen,upregulated.regen,upregulated.wound,msigdb.stress,downregulated.stress
downregulated.wound,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
upregulated.stress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
downregulated.regen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
msigdb.wound,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
msigdb.regen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
upregulated.regen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
upregulated.wound,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
msigdb.stress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
downregulated.stress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
pvals.to_csv("STRING_matched_pvals_hallmark_9by9.csv")

In [47]:
LogFC = np.zeros(shape=(len(hp_genesets), len(hallmark)))
for i in range(len(hp_genesets)):
    for j in range(len(hallmark)):
        test = test_dist[i][j]
        matched =test_matched_dist[i][j]
        LogFC[i][j] = math.log((test/matched))

In [48]:
LFC = pd.DataFrame(LogFC, columns = hallmark.keys(), index = hp_genesets.keys())

In [49]:
LFC

,downregulated.wound,upregulated.stress,downregulated.regen,msigdb.wound,msigdb.regen,upregulated.regen,upregulated.wound,msigdb.stress,downregulated.stress
downregulated.wound,-0.456302,-0.430930,-0.444344,-0.461382,-0.455965,-0.445419,-0.446807,-0.443209,-0.430166
upregulated.stress,-0.429390,-0.431654,-0.425666,-0.432769,-0.435765,-0.427884,-0.431454,-0.429034,-0.430617
downregulated.regen,-0.439494,-0.423205,-0.438724,-0.446358,-0.446038,-0.435629,-0.436036,-0.431246,-0.421916
msigdb.wound,-0.470631,-0.441099,-0.459247,-0.506833,-0.489331,-0.459120,-0.461460,-0.464184,-0.437737
msigdb.regen,-0.464669,-0.445462,-0.459681,-0.486989,-0.502304,-0.457428,-0.462616,-0.466620,-0.441835
upregulated.regen,-0.441229,-0.425421,-0.436002,-0.446506,-0.444694,-0.438290,-0.438075,-0.433768,-0.426502
upregulated.wound,-0.441639,-0.428200,-0.435759,-0.446730,-0.448417,-0.437081,-0.441736,-0.435562,-0.428867
msigdb.stress,-0.459749,-0.444516,-0.450133,-0.476164,-0.478128,-0.453831,-0.457693,-0.473261,-0.443943
downregulated.stress,-0.421614,-0.424523,-0.418188,-0.419820,-0.423534,-0.421992,-0.425037,-0.419397,-0.430469


In [50]:
LFC.to_csv("STRING_matched_LFC_hallmark_9by9.csv")

In [51]:
pd.DataFrame(test_dist)

,0,1,2,3,4,5,6,7,8
0,1.863501,1.918239,1.892708,1.849380,1.847338,1.891838,1.887901,1.879078,1.922620
1,1.918239,1.920146,1.931540,1.909091,1.890152,1.929169,1.921176,1.911072,1.925350
2,1.892708,1.931540,1.901548,1.874628,1.863423,1.908672,1.906556,1.899558,1.937587
3,1.849380,1.909091,1.874628,1.784741,1.801920,1.877734,1.872591,1.855436,1.918321
4,1.847338,1.890152,1.863423,1.801920,1.763095,1.868815,1.857857,1.835054,1.900133
5,1.891838,1.929169,1.908672,1.877734,1.868815,1.905834,1.905010,1.897608,1.930624
6,1.887901,1.921176,1.906556,1.872591,1.857857,1.905010,1.894760,1.890168,1.923153
7,1.879078,1.911072,1.899558,1.855436,1.835054,1.897608,1.890168,1.851797,1.916416
8,1.922620,1.925350,1.937587,1.918321,1.900133,1.930624,1.923153,1.916416,1.917051


In [52]:
pd.DataFrame(test_matched_dist)

,0,1,2,3,4,5,6,7,8
0,2.941028,2.951569,2.951615,2.933607,2.914537,2.953432,2.951380,2.927036,2.956053
1,2.947030,2.956644,2.956432,2.942901,2.922450,2.959359,2.957640,2.934974,2.961585
2,2.937334,2.949167,2.948782,2.929315,2.910873,2.950683,2.948613,2.923749,2.954588
3,2.960865,2.967520,2.967315,2.962716,2.939335,2.971853,2.970656,2.951470,2.971858
4,2.940015,2.950928,2.950859,2.932460,2.913557,2.952738,2.950694,2.926168,2.955768
5,2.941082,2.952080,2.951783,2.934600,2.915374,2.954146,2.952234,2.928125,2.957502
6,2.936166,2.948031,2.947796,2.927219,2.909091,2.949301,2.947119,2.921882,2.953034
7,2.975852,2.980765,2.979496,2.987044,2.960037,2.987466,2.987266,2.972541,2.987390
8,2.930880,2.943594,2.943595,2.919087,2.902166,2.944194,2.941744,2.914953,2.948383
